Proceso del Archivo SteamGames

In [1]:
import pandas as pd

df = pd.read_json('SteamGames.json', lines=True)
df = df.dropna(how='all')
df.rename(columns={'id': 'item_id'}, inplace=True)

def convert_to_float_or_zero(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return 0

df['price'] = df['price'].apply(convert_to_float_or_zero)
df['early_access'] = df['early_access'].replace({0.0: False, 1.0: True})
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_date'] = df['release_date'].replace('Soon..', pd.NaT)
df['item_id'] = df['item_id'].fillna(0).astype('int64')

df.to_csv('SteamGames.csv', index=False)
df.to_parquet('SteamGames.parquet', index=False)


Proceso del Archivo UserReviews

In [10]:
import json
import pandas as pd
import re
from textblob import TextBlob  # asegurarse de tener instalada la biblioteca TextBlob previamente

# Función para corregir y cargar cada línea del archivo 'australian_user_reviews.json' original
def cargar_y_corregir_linea(linea):
    # Reemplaza las comillas simples con dobles y corrige valores booleanos
    linea_corregida = linea.replace("'", '"').replace('True', 'true').replace('False', 'false')
    return json.loads(linea_corregida)

# Lista para almacenar los datos desanidados
data_list = []

# Expresión regular para extraer números de la columna 'funny'
numero_regex = re.compile(r'\d+')

# Lee el archivo 'australian_user_reviews.json' original, corrige y procesa cada línea
with open('UserReviews.json', 'r', encoding='utf-8') as archivo_json:
    for linea in archivo_json:
        try:
            entrada = cargar_y_corregir_linea(linea)
            user_id = entrada['user_id']
            user_url = entrada['user_url']
            # Iteración sobre cada reseña
            for reseña in entrada['reviews']:
                # Extrae el número de la columna 'funny'
                funny_valor = re.search(numero_regex, reseña.get('funny', ''))
                if funny_valor:
                    funny = int(funny_valor.group())
                else:
                    funny = None
                
                # Elimina 'Posted' de la columna 'posted'
                posted = reseña.get('posted', '').replace('Posted ', '', 1)

                reseña_dict = {
                    'user_id': user_id,
                    'user_url': user_url,
                    'funny': funny,
                    'posted': posted,
                    'item_id': int(reseña.get('item_id', '')),  # Convierte a entero
                    'helpful': reseña.get('helpful', ''),
                    'recommend': reseña.get('recommend', ''),
                    'review': reseña.get('review', '')  # Mantiene el texto original de la review
                }
                data_list.append(reseña_dict)
        except json.JSONDecodeError as e:
            None

# Crea el DataFrame
df = pd.DataFrame(data_list)

# Elimina la columna 'last_edited' si existe
if 'last_edited' in df.columns:
    df.drop(columns=['last_edited'], inplace=True) 

# Función para analizar el sentimiento de la reseña
def analyze_sentiment(review):
    if pd.isna(review) or not isinstance(review, str):
        return 1  # Sentimiento neutral para reseñas faltantes o no legibles
    else:
        # Se emplea TextBlob para calcular la polaridad del sentimiento
        polarity = TextBlob(review).sentiment.polarity
        if polarity < 0:
            return 0  # Sentimiento negativo
        elif polarity == 0:
            return 1  # Sentimiento neutral
        else:
            return 2  # Sentimiento positivo

# Aplica la función analyze_sentiment a cada reseña y crea la nueva columna 'sentiment_analysis'
df['sentiment_analysis'] = df['review'].apply(analyze_sentiment)

# Convierte la columna 'posted' a tipo datetime
df['posted'] = pd.to_datetime(df['posted'], errors='coerce')

# Elimina la columna original 'review'
df.drop(columns=['review'], inplace=True)

# Guarda el DataFrame modificado en un nuevo archivo CSV
df.to_csv('UserReviews.csv', index=False)

# Guarda el DataFrame modificado en un nuevo archivo Parquet
df.to_parquet('UserReviews.parquet', index=False)

print("El análisis de sentimiento ha sido  guardado en 'UserReviews.csv' y 'UserReviews.parquet'.")


C:\Users\mateo\AppData\Local\Temp\ipykernel_17912\3512060537.py:76: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['posted'] = pd.to_datetime(df['posted'], errors='coerce')


El análisis de sentimiento ha sido  guardado en 'UserReviews.csv' y 'UserReviews.parquet'.


In [11]:
import json
import pandas as pd

# Función para cargar y corregir cada línea del archivo JSON original
def cargar_y_corregir_linea(linea):
    # Cambia comillas simples por comillas dobles
    linea_corregida = linea.replace("'", '"')
    return json.loads(linea_corregida)

# Lista para almacenar los datos desanidados
data_list = []

# Lee el archivo JSON original, corrige y procesa cada línea
with open('UsersItems.json', 'r', encoding='utf-8') as archivo_json:
    for linea in archivo_json:
        try:
            entrada = cargar_y_corregir_linea(linea)
            # Obtén datos del usuario
            user_id = entrada['user_id']
            items_count = entrada['items_count']
            steam_id = entrada['steam_id']
            user_url = entrada['user_url']
            # Itera sobre cada ítem
            for item in entrada['items']:
                # Crea un diccionario para cada ítem y agrégalo a la lista
                item_dict = {
                    'user_id': user_id,
                    'items_count': items_count,
                    'steam_id': steam_id,
                    'user_url': user_url,
                    'item_id': item['item_id'],
                    'item_name': item['item_name'],
                    'playtime_forever': item['playtime_forever'],
                    'playtime_2weeks': item['playtime_2weeks']
                }
                data_list.append(item_dict)
        except json.JSONDecodeError:
            None

# Crea el DataFrame
df = pd.DataFrame(data_list)

# Guarda el DataFrame en un nuevo archivo CSV
df.to_csv('UsersItems.csv', index=False)

# Guarda el DataFrame en un nuevo archivo Parquet
df.to_parquet('UsersItems.parquet', index=False)

print("Se han guardado correctamente 'UsersItems.parquet' y 'UsersItems.csv'")


Se han guardado correctamente 'UsersItems.parquet' y 'UsersItems.csv'
